# PID control on TCLab

This notebook and the associated `pidgui.py` allows you to play with a very basic position form discrete PID using either a modelled version or the real Temperature control lab.

In [ ]:
from pidgui import PIDGUI

In [ ]:
%matplotlib notebook

In [ ]:
interface = PIDGUI()
interface.gui